In [0]:
   !pip install mecab-python3 -q

     |████████████████████████████████| 17.1MB 235kB/s 


In [0]:
# https://www.rondhuit.com/download.html Livedoorコーパスに基づいて作成した。
!gdown https://drive.google.com/uc?id=1O7Iym9RgyiiFGL-WZBiW_pwrhMmwcU_g

Downloading...
From: https://drive.google.com/uc?id=1O7Iym9RgyiiFGL-WZBiW_pwrhMmwcU_g
To: /content/train.tsv
15.2MB [00:00, 57.3MB/s]


In [0]:
import pandas as pd
import os
import numpy as np
import random 
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

seed_everything(99)


In [0]:
train_df = pd.read_csv("train.tsv",sep='\t')
train_df["category"] = train_df.label.astype('category').cat.codes
train_df.head()

,text,label,category
0,大島優子がここからどう破滅していくのか？ 『闇金ウシジマくん』特報解禁“闇金”という禁断の題...,movie-enter,4
1,インタビュー：クリスチャン・ベール「演じることができるのは役者だけ」公開当時、全米歴代2位と...,movie-enter,4
2,ブラックマジックデザイン、HyperDeck SSD レコーダーに タイムコード、DNxHD...,kaden-channel,2
3,センター試験終了！ 受験生ファンに眞鍋かをりが的確なアドバイスをしていた【話題】寒い週末、毎...,kaden-channel,2
4,彼女の香りは、柔軟剤の香り都内で一人暮らしをしているサナエさん（28歳・医療関連）は、同僚の...,dokujo-tsushin,0


# Baseline TF-IDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import TruncatedSVD
import MeCab
from sklearn.metrics import classification_report

In [0]:
m = MeCab.Tagger("-Owakati")

In [0]:
train_xs = train_df[:500]["text"].apply(lambda x: m.parse(x))
train_ys = train_df[:500]['category']

test_xs = train_df[2000:3000]["text"].apply(lambda x: m.parse(x))
test_ys = train_df[2000:3000]['category']

In [0]:
train_xs.head(5)

0    大島 優子 が ここ から どう 破滅 し て いく の か ？ 　 『 闇 金 ウシジマ ...
1    インタビュー ： クリスチャン ・ ベール 「 演じる こと が できる の は 役者 だけ...
2    ブラックマジックデザイン 、 HyperDeck SSD レコーダー に タイム コード 、...
3    センター 試験 終了 ！ 　 受験生 ファン に 眞 鍋 かをり が 的確 な アドバイス ...
4    彼女 の 香り は 、 柔軟 剤 の 香り 都内 で 一人暮らし を し て いる サナエ ...
Name: text, dtype: object

In [0]:
vectorizer = TfidfVectorizer()
train_dev_xs_ = vectorizer.fit_transform(train_xs)
test_xs_ = vectorizer.transform(test_xs)
print(train_dev_xs_.shape)

lsa = TruncatedSVD(300)
train_dev_xs_ = lsa.fit_transform(train_dev_xs_)
test_xs_ = lsa.transform(test_xs_)
print(train_dev_xs_.shape)


(500, 19401)
(500, 300)


In [0]:
model = GradientBoostingClassifier(random_state=23)
model.fit(train_dev_xs_, train_ys)
print(classification_report(test_ys, model.predict(test_xs_)))

              precision    recall  f1-score   support

           0       0.74      0.76      0.75       127
           1       0.74      0.69      0.72       118
           2       0.87      0.76      0.81       112
           3       0.32      0.54      0.40        69
           4       0.78      0.84      0.81       101
           5       0.78      0.60      0.68       126
           6       0.86      0.88      0.87       121
           7       0.96      0.83      0.89       132
           8       0.76      0.80      0.78        94

    accuracy                           0.75      1000
   macro avg       0.75      0.74      0.74      1000
weighted avg       0.78      0.75      0.76      1000



# FastText

## Default

In [0]:
from gensim.models.fasttext import FastText
import numpy as np

def sent2vec(model, sentence):
    vector = np.zeros(model.vector_size)
    for word in sentence:
        if word in model.wv.vocab:
            vector += model.wv.__getitem__(word)
    return vector / len(sentence)
    
paser = MeCab.Tagger()
token_list = [[token for token in sent.split()] for sent in train_xs]

In [0]:
fasttext_model = FastText(size=300)
fasttext_model.build_vocab(token_list)
fasttext_model.train(token_list, total_examples=fasttext_model.corpus_count,epochs=fasttext_model.epochs)

In [0]:
train_xs_ = np.array([sent2vec(fasttext_model, sent) for sent in train_xs])
test_xs_ = np.array([sent2vec(fasttext_model, sent) for sent in test_xs])
model = GradientBoostingClassifier(random_state=23)
model.fit(train_xs_, train_ys)
print(classification_report(test_ys, model.predict(test_xs_)))

              precision    recall  f1-score   support

           0       0.61      0.61      0.61       127
           1       0.35      0.34      0.34       118
           2       0.62      0.78      0.69       112
           3       0.21      0.19      0.20        69
           4       0.67      0.73      0.70       101
           5       0.30      0.31      0.30       126
           6       0.73      0.75      0.74       121
           7       0.50      0.48      0.49       132
           8       0.48      0.34      0.40        94

    accuracy                           0.52      1000
   macro avg       0.50      0.50      0.50      1000
weighted avg       0.51      0.52      0.51      1000



## 改善１：Epoch数調整

In [0]:
fasttext_model = FastText(size=300)
fasttext_model.build_vocab(token_list)
fasttext_model.train(token_list, total_examples=fasttext_model.corpus_count,epochs=20, random=23)

In [0]:
train_xs_ = np.array([sent2vec(fasttext_model, sent) for sent in train_xs])
test_xs_ = np.array([sent2vec(fasttext_model, sent) for sent in test_xs])
model = GradientBoostingClassifier(random_state=23)
model.fit(train_xs_, train_ys)
print(classification_report(test_ys, model.predict(test_xs_)))

              precision    recall  f1-score   support

           0       0.73      0.74      0.73       127
           1       0.57      0.49      0.53       118
           2       0.78      0.78      0.78       112
           3       0.34      0.35      0.35        69
           4       0.67      0.68      0.68       101
           5       0.44      0.50      0.47       126
           6       0.75      0.80      0.77       121
           7       0.68      0.64      0.66       132
           8       0.66      0.61      0.63        94

    accuracy                           0.63      1000
   macro avg       0.62      0.62      0.62      1000
weighted avg       0.64      0.63      0.63      1000



## 改善２：Window size調整

In [0]:
window_size = 8
fasttext_model = FastText(size=300,window=window_size)
fasttext_model.build_vocab(token_list)
fasttext_model.train(token_list, total_examples=fasttext_model.corpus_count,epochs=20,random=23)

In [0]:
train_xs_ = np.array([sent2vec(fasttext_model, sent) for sent in train_xs])
test_xs_ = np.array([sent2vec(fasttext_model, sent) for sent in test_xs])
model = GradientBoostingClassifier(random_state=23)
model.fit(train_xs_, train_ys)
print(classification_report(test_ys, model.predict(test_xs_)))

              precision    recall  f1-score   support

           0       0.76      0.74      0.75       127
           1       0.62      0.49      0.55       118
           2       0.81      0.81      0.81       112
           3       0.32      0.36      0.34        69
           4       0.69      0.76      0.72       101
           5       0.50      0.53      0.51       126
           6       0.76      0.83      0.80       121
           7       0.71      0.74      0.72       132
           8       0.76      0.60      0.67        94

    accuracy                           0.67      1000
   macro avg       0.66      0.65      0.65      1000
weighted avg       0.67      0.67      0.67      1000



## 改善３：名詞動詞のみ考慮する
ノーズ除去とWindow size拡大の効果がある。

In [0]:
def get_noun_verb(parser, doc, typ=["名詞", "動詞"]):
    parsed = parser.parse(doc).strip()
    results = parsed.split("\n")[:-1]
    word_list = []
    for parse_result in results:
        word = parse_result.split("\t")[0]
        result = parse_result.split("\t")[1].split(",")
        pos = result[0]
        origin = result[6]
        if pos in typ:
            if pos == "名詞":
                word_list.append(word)
            else:
                word_list.append(origin)
    return word_list

In [0]:
train_xs = train_df[:500]["text"].apply(lambda x: get_noun_verb(paser, x))
test_xs = train_df[2000:3000]["text"].apply(lambda x: get_noun_verb(paser, x))
token_list = [[token for token in sent] for sent in train_xs]

In [0]:
fasttext_model = FastText(size=300,window=window_size)
fasttext_model.build_vocab(token_list)
fasttext_model.train(token_list, total_examples=fasttext_model.corpus_count,epochs=20,random=23)

In [0]:
train_xs_ = np.array([sent2vec(fasttext_model, sent) for sent in train_xs])
test_xs_ = np.array([sent2vec(fasttext_model, sent) for sent in test_xs])
model = GradientBoostingClassifier(random_state=23)
model.fit(train_xs_, train_ys)
print(classification_report(test_ys, model.predict(test_xs_)))

              precision    recall  f1-score   support

           0       0.73      0.79      0.76       127
           1       0.69      0.56      0.62       118
           2       0.72      0.78      0.75       112
           3       0.37      0.46      0.41        69
           4       0.78      0.82      0.80       101
           5       0.69      0.62      0.65       126
           6       0.85      0.85      0.85       121
           7       0.81      0.88      0.84       132
           8       0.81      0.66      0.73        94

    accuracy                           0.73      1000
   macro avg       0.72      0.71      0.71      1000
weighted avg       0.73      0.73      0.73      1000



## 改善４：Stopwordsを除く

In [0]:
#@title
stopwords = """する
いる
れる
こと
ある
なる
ため
られる
よう
ころ
せる
できる
出来る
しまう
あそこ
あたり
あちら
あっち
あと
あな
あなた
あれ
いくつ
いつ
いま
いや
いろいろ
うち
おおまか
おまえ
おれ
がい
かく
かたち
かやの
から
がら
きた
くせ
ここ
こっち
こと
ごと
こちら
ごっちゃ
これ
これら
ごろ
さまざま
さらい
さん
しかた
しよう
すか
ずつ
すね
すべて
ぜんぶ
そう
そこ
そちら
そっち
そで
それ
それぞれ
それなり
たくさん
たち
たび
ため
だめ
ちゃ
ちゃん
てん
とおり
とき
どこ
どこか
ところ
どちら
どっか
どっち
どれ
なか
なかば
なに
など
なん
はじめ
はず
はるか
ひと
ひとつ
ふく
ぶり
べつ
へん
ぺん
ほう
ほか
まさ
まし
まとも
まま
みたい
みつ
みなさん
みんな
もと
もの
もん
やつ
よう
よそ
わけ
わたし
年
月
日
時
分
秒
週
火
水
木
金
土
国
都
道
府
県
市
区
町
村""".split("\n")

In [0]:
def get_noun_verb(parser, doc, typ=["名詞", "動詞"]):
    parsed = parser.parse(doc).strip()
    results = parsed.split("\n")[:-1]
    word_list = []
    for parse_result in results:
        word = parse_result.split("\t")[0]
        result = parse_result.split("\t")[1].split(",")
        pos = result[0]
        origin = result[6]
        if pos in typ:
            if pos == "名詞":
                word_list.append(word)
            else:
                word_list.append(origin)
    return [word for word in word_list if word not in stopwords] # ここでStopwordsを除く

In [0]:
train_xs = train_df[:500]["text"].apply(lambda x: get_noun_verb(paser, x))
test_xs = train_df[2000:3000]["text"].apply(lambda x: get_noun_verb(paser, x))
token_list = [[token for token in sent] for sent in train_xs]

In [0]:
fasttext_model = FastText(size=300, window=window_size)
fasttext_model.build_vocab(token_list)
fasttext_model.train(token_list, total_examples=fasttext_model.corpus_count,epochs=20,random=23)

In [0]:
train_xs_ = np.array([sent2vec(fasttext_model, sent) for sent in train_xs])
test_xs_ = np.array([sent2vec(fasttext_model, sent) for sent in test_xs])
model = GradientBoostingClassifier(random_state=23)
model.fit(train_xs_, train_ys)
print(classification_report(test_ys, model.predict(test_xs_)))

              precision    recall  f1-score   support

           0       0.87      0.71      0.78       127
           1       0.69      0.75      0.72       118
           2       0.83      0.81      0.82       112
           3       0.33      0.41      0.36        69
           4       0.76      0.79      0.78       101
           5       0.58      0.64      0.61       126
           6       0.91      0.82      0.86       121
           7       0.90      0.86      0.88       132
           8       0.79      0.78      0.78        94

    accuracy                           0.74      1000
   macro avg       0.74      0.73      0.73      1000
weighted avg       0.76      0.74      0.75      1000



## 改善５：SIF(smooth inverse frequency)でReweightする
SIF(smooth inverse frequency)とは、簡単に言うと、単語ベクトルの加重平均で文ベクトルを作る手法である。単語の出現する頻度が高いほどWeightが小さい。

In [0]:
#@title
# https://towardsdatascience.com/fse-2b1ffa791cf9
import numpy as np
REAL = np.float32 

def sif_embeddings(model,sentences,  alpha=1e-3):
    """Compute the SIF embeddings for a list of sentences
    Parameters
    ----------
    sentences : list
        The sentences to compute the embeddings for
    model : `~gensim.models.base_any2vec.BaseAny2VecModel`
        A gensim model that contains the word vectors and the vocabulary
    alpha : float, optional
        Parameter which is used to weigh each individual word based on its probability p(w).
    Returns
    -------
    numpy.ndarray 
        SIF sentence embedding matrix of dim len(sentences) * dimension
    """
    
    vlookup = model.wv.vocab  # Gives us access to word index and count
    vectors = model.wv        # Gives us access to word vectors
    size = model.vector_size  # Embedding size
    
    Z = 0
    for k in vlookup:
        Z += vlookup[k].count # Compute the normalization constant Z
    
    output = []
    
    # Iterate all sentences
    for s in sentences:
        count = 0
        v = np.zeros(size, dtype=REAL) # Summary vector
        # Iterare all words
        for w in s:
            if w in vlookup:
            # The loop over the the vector dimensions is completely unecessary and extremely slow
                v += ( alpha / (alpha + (vlookup[w].count / Z))) * vectors[w]
                count += 1
                
        if count > 0:
            for i in range(size):
                v[i] *= 1/count
        output.append(v)
    return np.vstack(output).astype(REAL)

In [0]:
train_xs = train_df[:500]["text"].apply(lambda x: get_noun_verb(paser, x))
test_xs = train_df[2000:3000]["text"].apply(lambda x: get_noun_verb(paser, x))

train_xs_ = sif_embeddings(fasttext_model, train_xs)
test_xs_ = sif_embeddings(fasttext_model, test_xs)

In [0]:

model = GradientBoostingClassifier(random_state=23)
model.fit(train_xs_, train_ys)
print(classification_report(test_ys, model.predict(test_xs_)))

              precision    recall  f1-score   support

           0       0.83      0.65      0.73       127
           1       0.72      0.73      0.73       118
           2       0.85      0.79      0.82       112
           3       0.37      0.61      0.46        69
           4       0.77      0.91      0.83       101
           5       0.70      0.60      0.65       126
           6       0.85      0.88      0.87       121
           7       0.93      0.84      0.88       132
           8       0.77      0.73      0.75        94

    accuracy                           0.75      1000
   macro avg       0.75      0.75      0.75      1000
weighted avg       0.77      0.75      0.76      1000



## 改善６：Further Training（Online training)
データが追加された時、もしくは、学習済みモデルにFine Tuningする時使う。

In [0]:
# 例えば、新しい500件の文書が追加された。
additional_data = train_df[500:1000]["text"].apply(lambda x: get_noun_verb(paser, x))
token_list = [[token for token in sent] for sent in additional_data]

In [0]:
# まずVocabularyをUpdateする必要がある。
fasttext_model.build_vocab(token_list, update=True)
#　その後は普通の学習
fasttext_model.train(token_list, total_examples=fasttext_model.corpus_count,epochs=20,random=23)

In [0]:
train_xs_ = sif_embeddings(fasttext_model, train_xs)
test_xs_ = sif_embeddings(fasttext_model, test_xs)
model = GradientBoostingClassifier(random_state=23)
model.fit(train_xs_, train_ys)
print(classification_report(test_ys, model.predict(test_xs_)))

              precision    recall  f1-score   support

           0       0.88      0.72      0.79       127
           1       0.64      0.70      0.67       118
           2       0.78      0.84      0.81       112
           3       0.49      0.54      0.51        69
           4       0.76      0.93      0.84       101
           5       0.81      0.67      0.74       126
           6       0.86      0.87      0.86       121
           7       0.91      0.85      0.88       132
           8       0.72      0.76      0.74        94

    accuracy                           0.77      1000
   macro avg       0.76      0.76      0.76      1000
weighted avg       0.78      0.77      0.77      1000



# Bertでやったらどうなる？
To Be Continued...